In [1]:

# Notebook Summary:

# V.History: 
# Date Last Modified: 14 May 2025

#--------------------------------------------------------------------------------------------------
'''
    This model transposes the uni-dimensional WQ dataset into a 2-dimensional dataset, including BNG 
    coordinates for spatio-temporal model construction.
    Date: March 25, 2025

    Following are the label encoders created:
        1. isComplianceSample            - encoder_isComplianceSample.pkl
        2. purpose_name                  - encoder_purpose_name.pkl
        3. determinand_unit_name         - encoder_determinand_unit_name.pkl

    This step enables the model to recognise compliance samples, their purposes, and the varied 
    unit names of Determinands.
'''
#--------------------------------------------------------------------------------------------------


#--------------------------------------------------------------------------------------------------
#Pre-Requisite : 
    #Kernel Python 3 (ipykernel) is required to run this notebook 
    #Required python version - Python 3.10.15 and its compatible Numpy , ScikitLearn libraries

#Old Name: 12_NB_10Transpose_2959_All2.ipynb
#--------------------------------------------------------------------------------------------------

#Proposed Validations cachements: 
#a. River Dee (Wales)
#b. Catchments in Ireland (potentially available through Teagasc)
#c. Skerne
#d. Browney
#e. Frome
#f. Wye

'''
    Intro Section
'''

'\n    Intro Section\n'

In [2]:
#Check python version compatibility 3.10 or above is required
!python -V
python_version=!(python --version 2>&1)
print (python_version)

Python 3.10.15
['Python 3.10.15']


In [3]:
ls ~/.local/share/jupyter/kernels

chem2/


In [4]:
#Check python version compatibility 3.10 or above is required

#sudo nano  ~/.local/share/jupyter/kernels/python311/kernel.json
#to display the system version of the pandas
#!pip show pandas
#to display this notebook's kernel version of the pandas
#%pip show pandas

import sys
sys.version

'3.10.15 | packaged by conda-forge | (main, Oct 16 2024, 01:24:24) [GCC 13.3.0]'

In [5]:
####################################################################################################
#Begin CARD (Transpose data for Phosphate and Phosphorus Family for 24 Years of WQ data for England)
####################################################################################################

In [6]:
%run "..//99_Common_Utils/99_NB_CommonUtils.ipynb" #Library Declaration section - Installing or Initiating all required Python Libraries

Intalling required libraries and utilities.....
Uses Python 3 (ipykernel) (Local)
Python 3.10.15
['Python 3.10.15']

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip in

2025-05-16 13:27:23.760685: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747402043.785280  107651 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747402043.793091  107651 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747402043.813587  107651 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747402043.813607  107651 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747402043.813610  107651 computation_placer.cc:177] computation placer alr


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
|| Completed intalling required libraries and utilities ||

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [7]:
'''
There are 52 individual units of measure in the WQ monitoring data and over 38 for the P-Family. Phosphate or Orthophosphate is 
measured in 'mg/l'. Considerations were made to convert these metrics into a single unit of 'mg/l' (aligned to P unit measure). 
Below are the 'Conversion functions' / code snippets for that. However, the conversion was later discarded in favour of relative weights for 
different metrics in spatio-temporal models. The code is kept here to document this experiment.

Notes about the conversion considerations:

1. Density Considerations: Some conversions, such as % w/w or mg/kg, assume the substance has a similar density to water (1 g/ml). 
If the density differs, adjust the conversion factor accordingly.

2. Context-Specific Units: Units like bq/l (radioactivity), cfu/0.1l (colony-forming units), us/cm (conductivity), 
and rfu (fluorescence) require additional context or calibration to convert meaningfully to mg/l.

3. Load Units: Units like kg/ann, t/d, and t/qtr are load measurements and depend on the total volume of water per time period for conversion.

The code provides a basic framework, but real-world applications may require more specific formulas and data, especially for 
context-specific units.


There are 38 Unique metric for Phosphate family
mg/l , coded, ug/l , % , mg/kg, ug/kg, m, ppm, ngr, unitless, phi, ng/l, min, deg, no/g, kg/d, 
deccafix, pres/nf, mw.s/sq. , no. , no/100ml, abs/cm, nm, bq/l, text, % v/v, kt/qtr, t/qtr, kg/qtr,
t/d, % w/w, yes/no, l/kg/h, cfu/0.1l, us/cm, g/l, t/wk, ug
'''

#-------------------------------------------------------------------------------------
#SAMPLE CONVERSION :-
# Uni directional data sample 
#Sl.No. | Determinand_name | Determinand_Definition | Result | Date
#  1      Oxygen             Disolved oxygen           5.2     1-1-2000
#  2      Nitrogen           Nitrogen oxyde            4.1     1-1-2000
#  3      Oxygen             Disolved oxygen           5.8     2-1-2000
#  4      Nitrogen           Nitrogen oxyde            4.9     2-1-2000
#  5      Oxygen             Disolved oxygen           5.8     1-1-2000
#  6      Nitrogen           Nitrogen oxyde            4.9     1-1-2000

# Converted 2 dimesional data Sample 
#Sl.No. | Oxygen | Nitrogen | Date
#  1      11.0        4.1     1-1-2000
#  1      9.0         4.1     1-1-2000
#-------------------------------------------------------------------------------------


'''
Metrics conversion
'''


'\nMetrics conversion\n'

In [8]:
# Conversion functions 29 Conversions
def first_convert_to_mg_per_l(value, unit, density=1.0):
    """
    Convert various units to mg/l.
    
    Parameters:
    - value: The numerical value to convert.
    - unit: The unit of the given value as a string.
    - density: Density of the substance in g/ml (default is 1.0, equivalent to water at room temperature).

    Returns:
    - The converted value in mg/l.
    """
    cUnit = 'mg/l'
    if unit == "mg/l":
        return value, cUnit
    elif unit == "%":
        # 1% = 10,000 mg/l
        return value * 10000, cUnit
    elif unit == "ug/l":
        # 1 ug/l = 0.001 mg/l
        return value * 0.001, cUnit
    elif unit == "mg/kg":
        # Assuming water density, 1 mg/kg = 1 mg/l
        return value, cUnit
    elif unit == "% v/v":
        # 1% v/v = 10,000 mg/l (for liquids like ethanol in water)
        return value * 10000, cUnit
    elif unit == "% w/w":
        # 1% w/w = 10,000 mg/l (assuming density is similar to water)
        return value * 10000, cUnit
    elif unit == "bq/l":
        # Conversion depends on the specific isotope (radioactivity)
        #raise ValueError("Conversion for 'bq/l' requires context-specific information.")
        return value, cUnit
    elif unit == "cfu/0.1l":
        # Colony forming units conversion needs microbiological context
        #raise ValueError("Conversion for 'cfu/0.1l' requires context-specific information.")
        return value, unit
    elif unit == "g/l":
        # 1 g/l = 1000 mg/l
        return value * 1000, cUnit
    elif unit == "kg/ann":
        # Annual load conversion requires volume flow data
        #raise ValueError("Conversion for 'kg/ann' requires annual volume flow data.")
        return value, unit
    elif unit == "kg/d":
        # Daily load conversion requires volume flow data
        #raise ValueError("Conversion for 'kg/d' requires daily volume flow data.")
        return value, unit
    elif unit == "kg/qtr":
        # Quarterly load conversion requires volume flow data
        #raise ValueError("Conversion for 'kg/qtr' requires quarterly volume flow data.")
        return value, unit
    elif unit == "kt/qtr":
        # Kilotons per quarter conversion requires volume flow data
        #raise ValueError("Conversion for 'kt/qtr' requires quarterly volume flow data.")
        return value, unit
    elif unit == "ug/kg":
        # Assuming water density, 1 ug/kg = 0.001 mg/l
        return value * 0.001, cUnit
    elif unit == "ng/l":
        # 1 ng/l = 1e-6 mg/l
        return value * 1e-6, cUnit
    elif unit == "no/100ml":
        # Conversion depends on the context of measurement
        #raise ValueError("Conversion for 'no/100ml' requires context-specific information.")
        return value, unit
    elif unit == "us/cm":
        # Conductivity conversion needs additional information about the substance
        #raise ValueError("Conversion for 'us/cm' requires substance-specific details.")
        return value, unit
    elif unit == "no/g":
        # Conversion for counts per gram needs context
        #raise ValueError("Conversion for 'no/g' requires context-specific information.")
        return value, unit
    elif unit == "ppm":
        # 1 ppm = 1 mg/l (assuming water density)
        return value * 1, cUnit
    elif unit == "t/ann":
        # Tons per annum conversion requires flow data
        #raise ValueError("Conversion for 't/ann' requires annual volume flow data.")
        return value, unit
    elif unit == "t/d":
        # Tons per day conversion requires flow data
        #raise ValueError("Conversion for 't/d' requires daily volume flow data.")
        return value, unit
    elif unit == "t/qtr":
        # Tons per quarter conversion requires flow data
        #raise ValueError("Conversion for 't/qtr' requires quarterly volume flow data.")
        return value, unit
    elif unit == "t/wk":
        # Tons per week conversion requires flow data
        #raise ValueError("Conversion for 't/wk' requires weekly volume flow data.")
        return value, unit
    elif unit == "ug":
        # Micrograms conversion needs volume data
        #raise ValueError("Conversion for 'ug' requires volume data.")
        return value, unit
    elif unit == "ng/kg":
        # Assuming water density, 1 ng/kg = 1e-6 mg/l
        return value * 1e-6, cUnit
    elif unit == "pg/l":
        # 1 pg/l = 1e-9 mg/l
        return value * 1e-9, cUnit
    elif unit == "pg/m3":
        # Conversion requires volume information
        #raise ValueError("Conversion for 'pg/m3' requires volume context.")
        return value, unit
    elif unit == "rfu":
        # Relative fluorescence units are context-specific
        #raise ValueError("Conversion for 'rfu' is context-specific and requires calibration data.")
        return value, unit
    else:
        #raise ValueError("Unknown unit: {}".format(unit))
        return value, unit

# Example usage
try:
    result, resultUnit = first_convert_to_mg_per_l(5, "%")
    print("Converted value: {} {}".format(result, resultUnit))
except ValueError as e:
    print("Error:", e)


Converted value: 50000 mg/l


In [9]:
def second_convert_to_mg_per_l(value, unit):
    """
    Converts various metrics to mg/L.
    Only applicable units are converted; others raise a ValueError.
    """
    unit = str(unit).lower()
    conversion_factors = {
        "mg/l": 1,            # Already in mg/L
        "%": 10_000,          # Assumes % w/v (1% = 10,000 mg/L)
        "ug/l": 1e-3,         # Micrograms per liter
        "mg/kg": 1,           # Assuming 1 kg/L density
        "% v/v": 10_000,      # Assumes % v/v in water
        "% w/w": 10_000,      # Assumes % w/w in water
        "g/l": 1_000,         # Grams per liter
        "ug/kg": 1e-3,        # Micrograms per kg (assuming 1 kg/L density)
        "ng/l": 1e-6,         # Nanograms per liter
        "ppm": 1,             # Equivalent to mg/L
        "bq/l": 1,            # Assumes radionuclide mass equivalent in mg/L
        "cfu/0.1l": 10,       # Colony-forming units per 0.1L to mg/L (arbitrary mass equivalence)
        "kg/d": 1 / 0.001,    # Assuming flow of 1 L/d
        "kg/qtr": 1 / (0.001 * 90),  # Quarterly conversion with 90 days assumed
        "kt/qtr": 1e6 / (0.001 * 90),# Kilotons per quarter
        "no/100ml": 10,       # Assuming 1 unit mass equivalent
        "us/cm": None,        # Cannot convert electrical conductivity to mg/L directly
        "no/g": None,         # Cannot convert counts to mg/L directly
        "t/d": 1e6 / 0.001,   # Tons per day assuming 1 L/d
        "t/qtr": 1e6 / (0.001 * 90),
        "t/wk": 1e6 / (0.001 * 7),
        "coded": None,        # Unsupported
        "m": None,            # Unsupported
        "ngr": None,          # Unsupported
        "unitless": None,     # Unsupported
        "phi": None,          # Unsupported
        "min": None,          # Unsupported
        "deg": None,          # Unsupported
        "deccafix": None,     # Unsupported
        "pres/nf": None,      # Unsupported
        "mw.s/sq.": None,     # Unsupported
        "no.": None,          # Unsupported
        "abs/cm": None,       # Unsupported
        "nm": None,           # Unsupported
        "text": None,         # Unsupported
        "yes/no": None,       # Unsupported
        "l/kg/h": None,       # Unsupported
        "ug": 1e-3,           # Micrograms to mg
    }

    if unit in conversion_factors and conversion_factors[unit] is not None:
        return value * conversion_factors[unit], unit
    else:
        #raise ValueError(f"Unsupported or non-convertible unit: {unit}")
        #raise ValueError(f"Unsupported")
        #raise ValueError(np.nan)
        return value, unit


# Example usage
try:
    #User input
    #value = float(input("Enter the value: "))
    #unit = input("Enter the unit: ")
    
    #Hardcoded values
    value = 1.1
    unit = 'ug'
    unit = 'text'
    result = second_convert_to_mg_per_l(value, unit)
    print(f"{value} {unit} is equal to {result} mg/L.")
except ValueError as e:
    print(e)


1.1 text is equal to (1.1, 'text') mg/L.


In [10]:
#STEP 1: Use the 24 years WQ data for England gathered for Phosphate & Phosphorus family in the above step
# To carry out the Transpose operation 
#==========================================================================================================

In [11]:
# Read Whole dataset
#folderpath = cleansed
#filename = '12_NB_10Transpose_2959_All1.csv'
#showtime()
#dfPSpatio_outer = loaddata(folderpath, filename, path = 'gcs://rdmai_dev_data/')
#showtime()


showtime()
#Bigger size [All types of Sample purpose Including Reactive monitoring] 
#dfPall_outer = pd.read_csv('gcs://rdmai_dev_data/cleansed/12_NB_10Transpose_2959_All1_With_R_Mon.csv' , #Commented 11 May 2025
dfPall_outer = pd.read_csv('gcs://rdmai_dev_data/cleansed/07_nb_transpose_2959_all1_with_r_mon.csv',     #Added 11 May 2025
                           usecols = ['sampleDateTime', 'samplingPoint_name',
                                      'determinand_notation', 'samplingPoint_notation',
                                      'samplingPoint_easting', 'samplingPoint_northing',
                                      #'sampledMaterialType_name', This is unique as of now, hence not required to load this column
                                      'isComplianceSample', 'purpose_name',
                                      'determinand_unit_name', 'result'])
#file_to_write = '12_NB_10Transpose_2959_All2_With_R_Mon.csv'
file_to_write = '07_nb_transpose_2959_all1_with_r_mon.csv'

'''
#Smaller size [All types of Sample purpose Except Reactive monitoring]
#dfPall_outer = pd.read_csv('gcs://rdmai_dev_data/cleansed/12_NB_10Transpose_2959_All1_Without_R_Mon.csv' , #Commented 11 May 2025
dfPall_outer = pd.read_csv('gcs://rdmai_dev_data/cleansed/07_nb_transpose_2959_all1_without_r_mon.csv',     #Added 11 May 2025
                           usecols = ['sampleDateTime', 'samplingPoint_name',
                                      'determinand_notation', 'samplingPoint_notation',
                                      'samplingPoint_easting', 'samplingPoint_northing',
                                      #'sampledMaterialType_name', This is unique as of now, hence not required to load this column
                                      'isComplianceSample', 'purpose_name',
                                      'determinand_unit_name', 'result'])
#file_to_write = '12_NB_10Transpose_2959_All2_Without_R_Mon.csv'
file_to_write = '07_nb_transpose_2959_all1_without_r_mon.csv'
'''
showtime()
print(file_to_write)

print(dfPall_outer["determinand_notation"].nunique(), dfPall_outer["determinand_unit_name"].nunique(), 
      dfPall_outer["isComplianceSample"].nunique(), dfPall_outer["purpose_name"].nunique())

#With R_Mon - Inclusive -> 1891 68 2 26
print(len(dfPall_outer))


dfPall_outer["isComplianceSample"].value_counts()

16 May 2025 13:27:38
16 May 2025 13:31:25
07_nb_transpose_2959_all1_with_r_mon.csv
1891 68 2 26
33424159


isComplianceSample
False    33166122
True       258037
Name: count, dtype: int64

In [12]:
#function to save encldoed labels into a pickle file
def train_and_save_encoders(data, categorical_columns, save_path='encoders'):
    """
    Train and save individual label encoders for each categorical column
    """
    import joblib
    encoders = {}
    save_path = "..//03_prediction_model/pkls"
   
    for col in categorical_columns:
        # Create and fit encoder
        le = LabelEncoder()
        data[col+"_label"] = le.fit_transform(data[col])
       
        # Save encoder to individual file
        encoder_path = f'{save_path}/encoder_{col}.pkl'
        joblib.dump(le, encoder_path)
        encoders[col] = encoder_path
       
    return data, encoders

In [13]:
from sklearn.preprocessing import LabelEncoder 
labelencoder= LabelEncoder() #initializing an object of class LabelEncoder

'''
Commented on 15 May 2025
dfPall_outer['isComplianceSample_label'] = labelencoder.fit_transform(dfPall_outer['isComplianceSample']) #fitting and transforming the desired categorical column.
dfPall_outer['purpose_name_label'] = labelencoder.fit_transform(dfPall_outer['purpose_name']) #fitting and transforming the desired categorical column.
dfPall_outer['determinand_unit_name_label'] = labelencoder.fit_transform(dfPall_outer['determinand_unit_name']) #fitting and transforming the desired categorical column.
'''

#Save Encoders into Pickle file
# List of categorical columns to encode
categorical_cols = ['isComplianceSample', 'purpose_name', 'determinand_unit_name']

# Process data and save encoders
dfPall_outer, encoder_paths = train_and_save_encoders(dfPall_outer, categorical_cols)

#dfPall_outer = dfPall_outer.drop(columns=['isComplianceSample', 'purpose_name', 'determinand_unit_name'])

print(dfPall_outer["determinand_notation"].nunique(), dfPall_outer["determinand_unit_name_label"].nunique(), 
      dfPall_outer["isComplianceSample_label"].nunique(), dfPall_outer["purpose_name_label"].nunique())

dfPall_outer.head(2)
print(len(dfPall_outer))

1891 68 2 26
33424159


In [14]:
dfPall_outer_refCompSample = dfPall_outer.drop_duplicates(subset=['isComplianceSample_label'])[['isComplianceSample','isComplianceSample_label']]
dfPall_outer_refPurpose = dfPall_outer.drop_duplicates(subset=['purpose_name_label'])[['purpose_name','purpose_name_label']]
dfPall_outer_refUnitName = dfPall_outer.drop_duplicates(subset=['determinand_unit_name_label'])[['determinand_unit_name','determinand_unit_name_label']]
print("\nTypes of complinace : ", len(dfPall_outer_refCompSample), 
      "\nPurpose Types       : ", len(dfPall_outer_refPurpose), 
      "\nVaried Unit names   : ", len(dfPall_outer_refUnitName))

#Types of complinace :  2 
#Purpose Types       :  26 
#Varied Unit names   :  68


Types of complinace :  2 
Purpose Types       :  26 
Varied Unit names   :  68


In [15]:
#if (file_to_write == "12_NB_10Transpose_2959_All2_With_R_Mon.csv"):
if (file_to_write == "07_nb_transpose_2959_all1_with_r_mon.csv"):
    '''
    savedata(dfPall_outer_refCompSample, "08_nb_transpose_2959_all2_UniqComplianceSamples.csv", 'gcs://rdmai_dev_data/NW_Final/')
    savedata(dfPall_outer_refPurpose, "08_nb_transpose_2959_all2_UniqPurposeNames.csv", 'gcs://rdmai_dev_data/NW_Final/')
    savedata(dfPall_outer_refUnitName, "08_nb_transpose_2959_all2_UniqDetermndUnitNames.csv", 'gcs://rdmai_dev_data/NW_Final/')
    '''
    savedata(dfPall_outer_refCompSample, "08_nb_transpose_2959_all2_UniqComplianceSamples.csv", 'gcs://rdmai_dev_data/cleansed/')
    savedata(dfPall_outer_refPurpose, "08_nb_transpose_2959_all2_UniqPurposeNames.csv", 'gcs://rdmai_dev_data/cleansed/')
    savedata(dfPall_outer_refUnitName, "08_nb_transpose_2959_all2_UniqDetermndUnitNames.csv", 'gcs://rdmai_dev_data/cleansed/')

#dfPall_outer_refCompSample.head(2)
#dfPall_outer_refPurpose.head(2)
#dfPall_outer_refUnitName.head(2)

saved, Location:  gcs://rdmai_dev_data/NW_Final/cleansed/08_nb_transpose_2959_all2_UniqComplianceSamples.csv
saved, Location:  gcs://rdmai_dev_data/NW_Final/cleansed/08_nb_transpose_2959_all2_UniqPurposeNames.csv
saved, Location:  gcs://rdmai_dev_data/NW_Final/cleansed/08_nb_transpose_2959_all2_UniqDetermndUnitNames.csv


In [16]:
dfPall_outer = dfPall_outer.drop(columns=['isComplianceSample', 'purpose_name', 'determinand_unit_name'])

In [17]:
dfPall_outer['sampleDateOnly'] = pd.to_datetime(dfPall_outer['sampleDateTime'])
dfPall_outer['sampleDateOnly'] = dfPall_outer['sampleDateOnly'].dt.date
dfPall_outer = dfPall_outer.set_index('sampleDateOnly')

dfPall_outer = dfPall_outer.drop(columns=['sampleDateTime'])

dfPall_outer.head(2)

,samplingPoint_notation,samplingPoint_name,determinand_notation,result,samplingPoint_easting,samplingPoint_northing,isComplianceSample_label,purpose_name_label,determinand_unit_name_label
sampleDateOnly,,,,,,,,,
2000-01-24,AN-01M04,R.OUSE A422 RD.BR.BRACKLEY,180,0.10,459427,236819,0,3,28
2000-01-24,AN-01M04,R.OUSE A422 RD.BR.BRACKLEY,6051,160.00,459427,236819,0,3,64


In [18]:
print("Total rows: ", len(dfPall_outer))
# Total rows: 33424159

Total rows:  33424159


In [19]:
#Commented on March 26 2025 as it was decided to drop (units) metrics standardization

#dfPall_outer_temp1 = dfPall_outer_temp[dfPall_outer_temp['determinand_unit_nameC'] == 'mg/l']
#print(len(dfPall_outer_temp), len(dfPall_outer_temp1))
#
#dfPall_result = dfPall_outer_temp.groupby(['sampleDateOnly', 'samplingPoint_notation',
#                                           'samplingPoint_easting', 'samplingPoint_northing',
#                                           'determinand_notation', 'determinand_unit_nameC']
#                                         ).agg(daily_avg=('resultC', 'mean')
#                                              ).reset_index()
#
print("\nTotal rows before Grouping by : ", len(dfPall_outer))
dfPall_result = dfPall_outer.groupby(['sampleDateOnly', 'samplingPoint_notation',
                                      'samplingPoint_easting', 'samplingPoint_northing',
                                      'isComplianceSample_label',
                                      'purpose_name_label',
                                      'determinand_unit_name_label', 
                                      'determinand_notation']
                                    ).agg(daily_avg=('result', 'mean')
                                         ).reset_index()
dfPall_result.head(2)
print("\nTotal rows after  Grouping by : ", len(dfPall_result))

'''
    Total rows before Grouping by :  33424159
    Total rows after  Grouping by :  32426089
'''


Total rows before Grouping by :  33424159

Total rows after  Grouping by :  32426089


'\n    Total rows before Grouping by :  33424159\n    Total rows after  Grouping by :  32426089\n'

In [20]:
'''
dfPall_result['sampleDateOnly'] = pd.to_datetime(dfPall_result['sampleDateOnly'])
dfPall_result['sampleDateOnly'] = dfPall_result['sampleDateOnly'].dt.date
dfPall_result = dfPall_result.set_index('sampleDateOnly')

dfPall_result.head(2)
'''

"\ndfPall_result['sampleDateOnly'] = pd.to_datetime(dfPall_result['sampleDateOnly'])\ndfPall_result['sampleDateOnly'] = dfPall_result['sampleDateOnly'].dt.date\ndfPall_result = dfPall_result.set_index('sampleDateOnly')\n\ndfPall_result.head(2)\n"

In [21]:
print ("Total rows after  Grouping by : ", len(dfPall_result), 
       "\nTotal Determinands          : ", len(dfPall_result['determinand_notation'].unique()))

print(dfPall_result[['samplingPoint_easting', 'samplingPoint_northing']].nunique(),
      dfPall_result['samplingPoint_notation'].nunique())

#With R_Mon - Inclusive -> 
     #32426089 1891
     #samplingPoint_easting     19378
     #samplingPoint_northing    19966

#Without R_Mon - Inclusive -> 
     #31885281 1637
     #samplingPoint_easting     18694
     #samplingPoint_northing    19226
     #samplingPoint_notation.   21867

Total rows after  Grouping by :  32426089 
Total Determinands          :  1891
samplingPoint_easting     19378
samplingPoint_northing    19966
dtype: int64 23083


In [22]:
#Delete in memory datasets since below 2-D conversion results in error: 
#      'MemoryError: Unable to allocate 308. GiB for an array with shape (13196325, 3131) and data type float64

del(dfPall_outer)


In [23]:
'''
print(type(dfPall_result.index))
dfPall_result.index = pd.to_datetime(dfPall_result.index)
print(type(dfPall_result.index))
dfPall_result.head(2)
'''

'\nprint(type(dfPall_result.index))\ndfPall_result.index = pd.to_datetime(dfPall_result.index)\nprint(type(dfPall_result.index))\ndfPall_result.head(2)\n'

In [24]:
# Based on Nicolai & Lila's input, removing rows for the determinands that are of less than 100 observations over 24 years
print("Before Removing rows that has <100 observations recorded per Determinand : ", len(dfPall_result))
#dfPall_result = dfPall_result[dfPall_result['determinand_notation'].apply(lambda x: dfPall_result['determinand_notation'].value_counts() [x]>=100).values]

colName = "determinand_notation"
holdGT = 100
vCounts = dfPall_result['determinand_notation'].value_counts()
valid_Determinands_lst = vCounts[vCounts >= holdGT].index
dfPall_result = dfPall_result[dfPall_result[colName].isin(valid_Determinands_lst)]

print("After Removal............................................................: ", len(dfPall_result))

dfPall_result.head(2)

#With R_Mon - Inclusive -> 
       #Before removal: 32426089
       #After removal:  32411236
       #Difference:        14853

#Without R_Mon - Inclusive -> 
       #Before removal: 31885281
       #After removal:  31871766
       #Difference:        13515

Before Removing rows that has <100 observations recorded per Determinand :  32426089
After Removal............................................................:  32411236


,sampleDateOnly,samplingPoint_notation,samplingPoint_easting,samplingPoint_northing,isComplianceSample_label,purpose_name_label,determinand_unit_name_label,determinand_notation,daily_avg
0,2000-01-01,AN-70561033,489852,289360,0,24,28,99,8.70
1,2000-01-01,AN-70561033,489852,289360,0,24,28,111,13.00


In [25]:
#Transposint to two dimensional and fill na values with zero
result_transposed_Full = dfPall_result.pivot(index=['sampleDateOnly', 'samplingPoint_notation',
                                                    'samplingPoint_easting', 'samplingPoint_northing',
                                                    'determinand_unit_name_label', 
                                                    'purpose_name_label', 
                                                    'isComplianceSample_label',
#                                                    'determinand_unit_name'
                                                   ],
                                             columns=['determinand_notation'],
                                             values='daily_avg').fillna(0)

print("Total Records Before Transpose:  ", len(dfPall_result), 
      "After.........................:  ", len(result_transposed_Full))
result_transposed_Full.head(2)

#With R_Mon - Inclusive -> 
       #Total Records Before Transpose:  32411236
       #Total Records After Transpose:    9278441, 1773958

#Without R_Mon - Inclusive -> 
       #Total Records Before Transpose:  31871766
       #Total Records After Transpose:    9106238

Total Records Before Transpose:   32411236 After.........................:   9278441


determinand_notation                                                                                                                                        3     \
sampleDateOnly samplingPoint_notation samplingPoint_easting samplingPoint_northing determinand_unit_name_label purpose_name_label isComplianceSample_label         
2000-01-01     AN-70561033            489852                289360                 28                          24                 0                         0.00   
                                                                                   54                          24                 0                         0.00   

determinand_notation                                                                                                                                        4     \
sampleDateOnly samplingPoint_notation samplingPoint_easting samplingPoint_northing determinand_unit_name_label purpose_name_label isComplianceSample_label         
2000-01-01     AN-70561033            489852                289360                 28                          24                 0                         0.00   
                                                                                   54                          24                 0                         0.00   

determinand_notation                                                                                                                                        6     \
sampleDateOnly samplingPoint_notation samplingPoint_easting samplingPoint_northing determinand_unit_name_label purpose_name_label isComplianceSample_label         
2000-01-01     AN-70561033            489852                289360                 28                          24                 0                         0.00   
                                                                                   54                          24                 0                         0.00   

determinand_notation                                                                                                                                        28    \
sampleDateOnly samplingPoint_notation samplingPoint_easting samplingPoint_northing determinand_unit_name_label purpose_name_label isComplianceSample_label         
2000-01-01     AN-70561033            489852                289360                 28                          24                 0                         0.00   
                                                                                   54                          24                 0                         0.00   

determinand_notation                                                                                                                                        30    \
sampleDateOnly samplingPoint_notation samplingPoint_easting samplingPoint_northing determinand_unit_name_label purpose_name_label isComplianceSample_label         
2000-01-01     AN-70561033            489852                289360                 28                          24                 0                         0.00   
                                                                                   54                          24                 0                         0.00   

determinand_notation                                                                                                                                        31    \
sampleDateOnly samplingPoint_notation samplingPoint_easting samplingPoint_northing determinand_unit_name_label purpose_name_label isComplianceSample_label         
2000-01-01     AN-70561033            489852                289360                 28                          24                 0                         0.00   
                                                                                   54                          24                 0                         0.00   

determinand_notation                                      

In [26]:
del (dfPall_result)

In [27]:

'''
    Initially, the kernel struggled with handling large volumes of 9+ million rows with thousands of columns, 
    resulting in errors. To address this, a stratified sample of 40% (approximately 3.7 million rows) was 
    tested, which worked well. After consulting with a GCP infrastructure expert, additional compute resources 
    were added to handle such volume processing, and the below code was commented out.
    Pasupathi N, Lead Data Scientist, Cognizant - UK.
'''

'''
print (len(result_transposed_Full))
result_transposed_Full = result_transposed_Full.sample(frac=0.4,random_state=1).reset_index(drop=False)
print (len(result_transposed_Full))
result_transposed_Full.head(2)

#With R_Mon - Inclusive -> 
    #Stratified sample of 3711376 records taken out of the total 9278441 records, 
'''

'''
    Stratified sampling process to handle large volume transactions.
'''

'\n    Stratified sampling process to handle large volume transactions.\n'

In [28]:
print(file_to_write)

07_nb_transpose_2959_all1_with_r_mon.csv


In [29]:
'''
    Due to a GCP error in handling large volume files, the dataset was written into GCP for interim purposes. 
    Otherwise, the following steps resulted in an error, requiring a kernel restart and causing loss of 
    datasets and memory variables, necessitating the execution of steps from the start cell.
'''

'''
showtime()
# Save the Transposed data - 24 Years of data Grouped and Transposed
savedata(result_transposed_Full, file_to_write, 'gcs://rdmai_dev_data/NW_Final/')
print("Saved file at: gcs://rdmai_dev_data/NW_Final/", file_to_write)
showtime()

#At this step - Pasu 7th Apr 2025
'''

'''
    Interim Write
'''

'\n    Interim Write\n'

In [ ]:
showtime()
# Save the Transposed data - 24 Years of data Grouped and Transposed
#result_transposed_Full.to_csv('../NW_DataPP/12_NB_10Transpose_2959_All2_With_R_Mon.csv', index=True)
'''
    if ( file_to_write == '12_NB_10Transpose_2959_All2_Without_R_Mon.csv'):
        file_to_write  = '12_NB_10Transpose_2959_All2_Without_R_MonLess_UL_C.csv'
    else:
        file_to_write  = '12_NB_10Transpose_2959_All2_With_R_MonLess_UL_C.csv'
'''

if ( file_to_write == '07_nb_transpose_2959_all1_with_r_mon.csv'): 
    file_to_write  = '08_nb_transpose_2959_all2_with_r_mon.csv'
else:
    file_to_write  = '08_nb_transpose_2959_all2_without_r_mon.csv'

print ("Total Rows in Transposed dataset: ", len(result_transposed_Full))

savedata(result_transposed_Full, file_to_write, 'gcs://rdmai_dev_data/')


showtime()

16 May 2025 13:34:43
Total Rows in Transposed dataset:  9278441
saved, Location:  gcs://rdmai_dev_data/cleansed/08_nb_transpose_2959_all2_with_r_mon.csv
16 May 2025 14:48:25


()

In [ ]:
print(len(result_transposed_Full.columns))
print(result_transposed_Full.columns)

966
Index([   3,    4,    6,   28,   30,   31,   37,   39,   50,   52,
       ...
       9943, 9951, 9952, 9959, 9978, 9979, 9989, 9990, 9992, 9993],
      dtype='int64', name='determinand_notation', length=966)


In [ ]:
#Release memory back to GCP computes
#del (dfPall_outer, dfPall_outer_temp, dfPall_outer_temp1, result_transposed_Full)
del (result_transposed_Full)
gc.collect()

21

In [ ]:
#STEP 1: Use the 24 years WQ data for England gathered for Phosphate & Phosphorus family in the above step
# To carry out the Transpose operation - End
#==========================================================================================================

In [ ]:
#dbutils.notebook.exit("End Workload - Scrip stopped")

In [37]:
#End CARD
#In line comments completed 10-May-2025

In [38]:
'''
    Theme: BB1C - Predicting Phosphate / Orthophosphate in UK Catchments Using AIML Models
    Pasupathi N, Lead Data Scientist, Cognizant - UK.
'''

'\n    Theme: BB1C - Predicting Phosphate / Orthophosphate in UK Catchments Using AIML Models\n    Pasupathi N, Lead Data Scientist, Cognizant - UK.\n'